In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1591773072281_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# read the data 
df = sqlContext.read.csv('s3a://logistic-data/1l.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- click: boolean (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- 

In [30]:
# select the columns 'site_id','site_domain','site_category','app_id','app_domain','app_category' and display them, 
# so that you can analyse the values as they were shown in the segment 

df.select(['site_id','site_domain','site_category','app_id','app_domain','app_category']).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+-------------+--------+----------+------------+
| site_id|site_domain|site_category|  app_id|app_domain|app_category|
+--------+-----------+-------------+--------+----------+------------+
|856e6d3f|   58a89a43|     f028772b|ecad2386|  7801e8d9|    07d7df22|
|e151e245|   7e091613|     f028772b|ecad2386|  7801e8d9|    07d7df22|
|e3c09f3a|   d262cf1e|     28905ebd|ecad2386|  7801e8d9|    07d7df22|
|0da94452|   248e439f|     50e219e0|ecad2386|  7801e8d9|    07d7df22|
|1fbe01fe|   f3845767|     28905ebd|ecad2386|  7801e8d9|    07d7df22|
|d9750ee7|   98572c79|     f028772b|ecad2386|  7801e8d9|    07d7df22|
|83a0ad1a|   5c9ae867|     f028772b|ecad2386|  7801e8d9|    07d7df22|
|85f751fd|   c4e18dd6|     50e219e0|98fed791|  d9b5648e|    0f2161f8|
|85f751fd|   c4e18dd6|     50e219e0|fc6dc16f|  5c5a694b|    0f2161f8|
|1fbe01fe|   f3845767|     28905ebd|ecad2386|  7801e8d9|    07d7df22|
|1fbe01fe|   f3845767|     28905ebd|ecad2386|  7801e8d9|    07d7df22|
|57fe1b20|   5b62659

In [31]:
# create 3 new columns in the datafeame named 'pub_id', 'pub_domain', 'pub_category' 
# these columns will have the data from 'site_id','site_domain','site_category'
# Use the withColumn method 

df = df.withColumn('pub_id',df['site_id'])
df = df.withColumn('pub_domain',df['site_domain'])
df = df.withColumn('pub_category',df['site_category'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# replace the values in the columns 'pub_id', 'pub_domain', 'pub_category' with the values 'app_id','app_domain','app_category'
# the replcement should only happen if the value in site_id is '85f751fd'
# you will need withColumn and when functions

from pyspark.sql.functions import when

df = df.withColumn("pub_id",when(df["pub_id"] == '85f751fd', df['app_id']).otherwise(df["pub_id"]))
df = df.withColumn("pub_domain",when(df["pub_id"] == '85f751fd', df['app_domain']).otherwise(df["pub_domain"]))
df = df.withColumn("pub_category",when(df["pub_id"] == '85f751fd', df['app_category']).otherwise(df["pub_category"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# drop all the columns which are now redundunt 

df = df.drop('app_category','site_id','site_domain','site_category','app_id','app_domain')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['click', 'C1', 'banner_pos', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'month', 'dayofweek', 'day', 'hour', 'y', 'pub_id', 'pub_domain', 'pub_category']

In [35]:
# check the frequency of each category in device_id. did you get a similar distribution to the one shown in the segment? 
# use groupBy and count

df.groupBy("device_id").count().sort(desc("count")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|device_id|count|
+---------+-----+
| a99f214a|82579|
| c357dbff|   62|
| 0f7c61dc|   51|
| afeffc18|   34|
| 936e92fb|   27|
| cef4c8cc|   12|
| 28dc8687|   11|
| d857ffbb|    9|
| 9af87478|    9|
| 02da5312|    8|
| 987552d1|    7|
| b09da1c4|    7|
| 3cdb4052|    6|
| 03559b29|    6|
| abab24a7|    5|
| 1168ce02|    5|
| adc390d0|    4|
| 2c112522|    4|
| bc8b6d57|    4|
| 243bd3ab|    4|
+---------+-----+
only showing top 20 rows

In [36]:
# combine device_id, device_ip and device_type to form a new column which can identify users. 
# name the new column 'user' with data from device_id

df = df.withColumn('user',df['device_id'] )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# In that column whereever the device_id is missing use a combination of device_ip and device_type
# You will have to use the withcolumn along with a when condition. 
# You will the combination of device id and device ip can be done using the following code. 
from pyspark.sql import functions as sf
# sf.concat(sf.col('device_ip'),sf.lit('-'), sf.col('device_model'))

# use this code inside a when condition

df = df.withColumn('user', 
                   when(df["device_id"] == 'a99f214a', sf.concat(sf.col('device_ip'),sf.lit('-'), sf.col('device_model')))
                   .otherwise(df["device_id"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# find the frequency of each category in the user column, save this frequency in a new coulumn in the data frame
# join this dataframe with the earlier df, make sure that when you join all the rows 
# form the original table need to be present in the final table as well.
# use the join table command

# Create a new dataframe with the counts frequency of each user. 
u_count = df.groupBy("user").count().sort(desc("count"))
u_count = u_count.withColumnRenamed('user','user_d')
u_count = u_count.withColumnRenamed('count','user_count')

# Join above created data framw with the orginal dataframe 
df_j1 = df.join(u_count, df.user==u_count.user_d, "left").drop('user_d')

# To check the transformation call the columns from df
df_j1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['click', 'C1', 'banner_pos', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'month', 'dayofweek', 'day', 'hour', 'y', 'pub_id', 'pub_domain', 'pub_category', 'user', 'user_count']

In [39]:
# combine user and hour column to make a new columns user-hour. 
df_j1 = df_j1.withColumn('user_hour', sf.concat(sf.col('user'),sf.lit('-'), sf.col('hour')))

# again add a columns with the frequency of occurance each user-hour combiation
# new data frome for frequency of user-hour combination
u_count_h = df_j1.groupBy("user_hour").count().sort(desc("count"))
u_count_h = u_count_h.withColumnRenamed('user_hour','user_hour_d')
u_count_h = u_count_h.withColumnRenamed('count','user_hour_count')

# join the new dataframe with df_j1
df_j2= df_j1.join(u_count_h, df_j1.user_hour==u_count_h.user_hour_d, "left").drop('user_hour_d')

# Check the columns to know that the trnaformation is done
df_j2.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['click', 'C1', 'banner_pos', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'month', 'dayofweek', 'day', 'hour', 'y', 'pub_id', 'pub_domain', 'pub_category', 'user', 'user_count', 'user_hour', 'user_hour_d', 'user_hour_count']

In [40]:
# drop all redundent columns

df=df_j2.drop( 'device_id', 'device_ip', 'device_model','user','user_hour','month')

# dropping month as it has only one value it does not contribute in predictions. 
# This was demonstrated by Jaidev

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# find the number of catagories in each columns 
# use the countdistinct method
# create a list of all columns and then loop through each column counting the distinct values in it. 

from pyspark.sql.functions import col, countDistinct

df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).show()

# here * instructs the agg function to accept whatever number of columns are inside the brakets. 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---+----------+-----------+----------------+----+---+---+---+---+---+---+---+---------+---+----+---+------+----------+------------+----------+---------------+
|click| C1|banner_pos|device_type|device_conn_type| C14|C15|C16|C17|C18|C19|C20|C21|dayofweek|day|hour|  y|pub_id|pub_domain|pub_category|user_count|user_hour_count|
+-----+---+----------+-----------+----------------+----+---+---+---+---+---+---+---+---------+---+----+---+------+----------+------------+----------+---------------+
|    2|  7|         7|          4|               4|1722|  8|  9|399|  4| 64|154| 60|        7| 10|  24|  2|  2837|      1331|          19|        45|             12|
+-----+---+----------+-----------+----------------+----+---+---+---+---+---+---+---+---------+---+----+---+------+----------+------------+----------+---------------+

In [42]:
# drop colums which have lot of dictinct catagories. 

df=df.drop('pub_id','pub_domain','C14','C17')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Algorithm Building

Earlier in the module, Jaidev has performed model building exercise. You can refer to his notebooks for help.

In [43]:
# import all the necessary libraries 

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# create a list of catagorical columns wich are represented by integers. these will not be sent through the one hot encoder

cCols = ['C1', 'banner_pos', 'device_type', 'device_conn_type']
cCols += ['C18','C19','C21','C15','C16']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# if you remember form the earlier model building exercise the column C20 has negative values. 
# replace the negative values with 0. you can use the same UDF made by Jaidev. 

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

posMapper = udf(lambda x: 0 if x < 0 else x)
df = df.withColumn('C20_1', posMapper(df['C20']))
df = df.withColumn("C20_1int", df['C20_1'].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# run the string indexer on the columns which are categorical but are represented by strings.
# adding a string on the end of the output column to make it easy to identify

stringCols = ['pub_category']
for c in stringCols:
    outCol = c + '_ix'
    si = StringIndexer(inputCol=c, outputCol=outCol)
    df = si.fit(df).transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
# add the resultant columns from string indexer into the catagorical columns list. 
# make sure that you have the right column name for C20 as it has beed processed separately. 

cCols+=['pub_category_ix','C20_1int']
cCols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['C1', 'banner_pos', 'device_type', 'device_conn_type', 'C18', 'C19', 'C21', 'C15', 'C16', 'pub_category_ix', 'C20_1int']

In [48]:
# run all the categorical columns through the one hot encoder 

ohe = OneHotEncoderEstimator(inputCols=cCols, outputCols=[c+'Enc' for c in cCols])
enc_model = ohe.fit(df)
df = enc_model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# use the vector assember to combine all the encoded columns and the continuous variable columns 

# add the neumerical columns to the list of encoded traning columns
trainCols = [c+'Enc' for c in cCols]
trainCols += ['hour', 'day', 'dayofweek','user_hour_count','user_count']

#configure the assembler object and use it to transform df
assembler = VectorAssembler(inputCols=trainCols, outputCol='features')
df = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# split the data into training and testing data set

df_train, df_test = df.randomSplit([0.7,0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# train model in the training dataset

# import the logistic model
from pyspark.ml.classification import LogisticRegression

# create the logistic model, configure it and call fit it to the data. 
lr = LogisticRegression(featuresCol='features', labelCol='y')
model = lr.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# use the model to predict the class labels of test data. 

result_test = model.evaluate(df_test)

result_test.recallByLabel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.9962709009984362, 0.020283199387677]

In [57]:
# Write the prediction data frame so that when you need it you can directly call that dataframe. No need to train again
# use your AWS account to create an S3 bucket 

# create an S3 buket in your aws account.write the data there.
result_test.predictions.coalesce(1).write.option("header", "true").parquet("s3a://logistic-data/testresult1.parquet")

# parquet format is necessary as few columns in the dataframe are in dense vector format and you cannot save them in csv format.
# coalesce here is used to make sure the file written a single file. If this is not set the written file will be inparts. 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

At this point you can terminate the m4.xlarge cluster and start a m4.large cluster then continue further. 